# 🧭 **Routing: Come Scegliere il Miglior Esperto (o Tool) in Base alla Domanda**

---

## 🔍 **Cos'è il Routing? Perché è Importante?**

Gli LLM sono potenti… ma non onniscienti!

* 🧠 Gli LLM sono **modelli linguistici** addestrati a **predire token**, non a interpretare **strutture tabellari** o decidere **quale strumento usare** per un dato input.
* 🧮 Se fornisci una tabella a un LLM, essa verrà trattata come una **singola lunga stringa**, causando perdita di struttura.

![alt](../images/routing_1.png)

* 🛠️ **Approcci alternativi**:

  * Query SQL su database strutturati 📊: Memorizzare i dati in un database SQL e lasciare che un LLM scriva query per interrogare il database
  * Tool calling per task specializzati 🧰: Far si che un LLM interagisca con funzioni personalizzate 

  * Possiamo utilizzare entrambi nella stssa applicazione ma per farlo dobbiamo utilizzare il **Routing**

🧠 **Routing** è il meccanismo per decidere, in automatico, **quale approccio utilizzare (interrogare un DB o usare un Tool)** usare per rispondere a una domanda in base alla sua natura e quindi inidirizzarci verso l'una o l'altra implementazione.

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

---

## 🏗️ Esempio: Tre Esperti LLM

Hai 3 LLM specializzati:

1. 🚗 **Automobili** – Domande su auto, motori, batterie
2. 🍽️ **Cibo e ristoranti** – Critico gastronomico
3. 💻 **Tecnologia** – Software, gadget, AI

Obiettivo: capire a **quale esperto instradare la domanda**.

In [2]:
car_template = """You are an expert in automobiles. You have extensive knowledge about car mechanics, models, and automotive technology.
You provide clear and helpful answers about cars.

Here is a question:
{query}"""

restaurant_template = """You are a knowledgeable foo critic and restaurant reviewer. You have a deep understanding of different cuisines, dining experiences, and what makes a great restaurant.
You provide clear and helpful answers about cars.

Here is a question:
{query}"""

technology_template = """You are a tech expert with in-depth knowledge of the latest gadgets, software, and technological trends.
You provide insightful and detailed answers about technology.

Here is a question:
{query}"""


---

Quando facciamo una domanda sul cibo, vogliamo chiedere all'esperto di cibo.

Dunque, in qualche modo, dobbiamo indirizzare la domanda all'LLM esperto in cibo.

Analogamente vale lo stesso ragionamento con gli altri due esperti LLM.

Un approccio consiste nell'utilizzare domande esemplificative come quelle sulle auto, sui ristoranti e sulla tecnologia.

Dopodichè, utilizzare un modello di embedding per creare gli emebddings di queste domande.

Dopo che abbiamo ottenuto gli embeddings, 3 per ogni categoria, possiamo utilizzarli nella nostra funzione esterna per calcolare la cosine similarity tra una query e tali embeddings. Prendiamo il massimo score di similarity delle tre categorie e di questi tre scores il massimo score. Sapremo quindi con quale embedding la query ha la maggiore similarity e quindi anche la categoria di tale embedding. Possiamo quindi instradare la query verso l'LLM esperto in tale categoria. 


## 🔢 🔀 Approccio 1 – Routing via **Embedding Similarity**

### 🛠️ Step 1: Crea esempi per ogni categoria

```text
Domande di esempio → embeddings → 3 vettori per categoria (auto, food, tech)
```

### 🧮 Step 2: Similarità Coseno

Confronta la **query in arrivo** con gli esempi tramite **cosine similarity** (`langchain.utils.math.cosine_similarity`):

```python
similarity = cosine_similarity(query_embedding, example_embedding)
```

Prendi il valore massimo per ciascuna categoria e instrada verso:

* `car_template` se max è in auto
* `restaurant_template` se max è in food
* `tech_template` se max è in tech

### ✅ Esempio

**Query:** “What’s the best way to improve my car’s battery life?”
🔁 Viene instradata a → `car_template` ✔️

In [4]:
car_questions = [
    "What is the difference between a sedan and a SUV?",
    "How does a hybrid car save fuel?",
    "What should I look for when buying a used car?"
]

restaurant_questions = [
    "What makes a five-star restaurant exceptional?",
    "How do I choose a good wine pairing for my meal?",
    "What are the key elements of French cuisine?"
]

technology_questions = [
    "What are the latest advancements in AI?",
    "How do I secure my home network against cyber threats?",
    "What should I consider when buying a new smartphone?"
]

In [5]:
embeddings = OpenAIEmbeddings()

car_question_embeddings = embeddings.embed_documents(car_questions)

restaurant_question_embeddings = embeddings.embed_documents(restaurant_questions)

technology_question_embeddings = embeddings.embed_documents(technology_questions)

In [6]:
from langchain_community.utils.math import cosine_similarity

def prompt_router(input):
    query_embedding = embeddings.embed_query(input['query'])
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity([query_embedding], restaurant_question_embeddings)[0]
    tech_similarity = cosine_similarity([query_embedding], technology_question_embeddings)[0]

    max_similarity = max(max(car_similarity), max(restaurant_similarity), max(tech_similarity))

    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    
    else:
        print("Using TECH")
        return PromptTemplate.from_template(restaurant_template)
    
input_query = {"query": "What's the best way to improve my cars's battery life?"}

prompt = prompt_router(input_query)




Using CAR


In [ ]:
# integriamo tale funzione in una chain

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router) # viene fatto .invoke() del prompt sulla query
    | ChatOpenAI()
    | StrOutputParser()
)

In [8]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"Identifying a good vintage wine at a restaurant can be a daunting task, but there are a few tips to keep in mind:\n\n1. Look for a reputable wine list: A good restaurant will typically have a well-curated wine list with a variety of options from different regions and vintages. Check for a diverse selection of wines and ask the sommelier for recommendations.\n\n2. Check the label: Look for wines from well-known and respected producers, as they are more likely to produce high-quality vintage wines. Also, check the year on the label to ensure that it is a good vintage.\n\n3. Ask about the wine's provenance: Inquire about how the wine has been stored and cared for, as proper storage conditions are crucial for preserving the quality of a vintage wine.\n\n4. Trust your palate: Ultimately, the best way to identify a good vintage wine is to trust your own taste buds. Don't be afraid to ask for a small taste before committing to a full glass or bottle.\n\nBy keeping these tips in mind, you can

Ok, per il nostro esempio ha funzionato bene.

Tuttavia ci possono essere molti casi limite, gli embeddings potrebbero essere non sufficienti.

**Un'altro modo è quello di far classificare direttamente la domanda a un LLM.**

---

## 🧠🔀 Approccio 2 – Routing via **LLM Classifier**

### 🧠 Prompt di Classificazione

Prompt semplice:

```
You are good at classifying questions.
Given a question, classify it as one of: car, restaurant, technology.
```

➡️ LLM risponde con `car`, `restaurant` o `technology`.

### 🧠 Vantaggi

* 🧠 **Miglior gestione dei casi limite**
* 🔧 **Più facile da mantenere**
* 💸 Usa modelli economici (es: `gpt-3.5-turbo`)
* ⚡ Veloce, stabile, accurato

### ✅ Esempio

**Query:** “What are the latest trends in electric cars?”
🔁 Classificata come `technology` (e non `car`!), quindi → `tech_template` ✔️

In [16]:
# creiamo un template di classificazione per il modello LLM

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

classification_prompt = """You are good at classify a question.
Given the user question below, classify it as either being about 'Car', 'Restaurant', or 'Technology'.

<If the question is about car mechanics, models, or automotive technology, classify it as 'Car'>
<If the question is about cuisines, dining experiences, or restaurant services, classify it as 'Restaurant'>
<If the question is about gadgets, software, or technological trends, classify it as 'Technology'>

<question>
{question}
</question>

Classification:"""

classification_template = PromptTemplate.from_template(classification_prompt)

classification_template.invoke({"question": "cio"})


StringPromptValue(text="You are good at classify a question.\nGiven the user question below, classify it as either being about 'Car', 'Restaurant', or 'Technology'.\n\n<If the question is about car mechanics, models, or automotive technology, classify it as 'Car'>\n<If the question is about cuisines, dining experiences, or restaurant services, classify it as 'Restaurant'>\n<If the question is about gadgets, software, or technological trends, classify it as 'Technology'>\n\n<question>\ncio\n</question>\n\nClassification:")

In [14]:
# chain 
classification_chain = classification_template | ChatOpenAI() | StrOutputParser()


In [17]:
input_query = {"question": "What are the latest trends in electric cars?"}

classification_chain.invoke(input_query)

'Technology'

In [21]:
# utilizziamo la classification chain nel prompt router
def prompt_router(input):
    classification = classification_chain.invoke({'question': input['query']})

    if classification == 'Car':
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Restaurant":
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)

    else:
        print("Unexpected classification:", classification)
        return None

 
input_query = {"query": "What are the latest trends in electric cars?"}

prompt = prompt_router(input_query)

Using TECHNOLOGY


In [22]:
# integriamo tale router in una chain

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [23]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"When identifying a good vintage wine at a restaurant, there are several factors to consider:\n\n1. Reputation of the winery: Research the winery and its reputation for producing high-quality wines. Look for well-known and reputable wineries that have a history of producing exceptional vintages.\n\n2. Vintage year: Pay attention to the vintage year of the wine, as this can greatly affect its quality. In general, wines from good vintage years tend to be more valuable and higher in quality.\n\n3. Label information: Look for detailed information on the label, such as the grape variety, region, and any special designations or classifications. Wines that are labeled with specific appellations or vineyard names are often indicative of higher quality.\n\n4. Price point: While price is not always indicative of quality, vintage wines tend to be more expensive due to their limited availability and aging potential. Be cautious of wines that are priced significantly lower than others of the same v

---

## 📊 Confronto dei Due Metodi

| ⚙️ Metodo           | ✅ Vantaggi                                  | ⚠️ Svantaggi               |
| ------------------- | ------------------------------------------- | -------------------------- |
| **Embedding-based** | ✨ Veloce con pochi esempi                   | 😓 Debole con edge cases   |
| **LLM Classifier**  | 🔍 Preciso, flessibile, facile da estendere | 💸 Richiede chiamata a LLM |

👉 **Consiglio pratico**: **Usa LLM classifier** – è la scelta migliore per routing generalizzato e stabile.

---

## 🔜 **Prossimi Step**

Nel prossimo modulo:

➡️ Userai il **Routing** per decidere tra:

* 📄 Retrieval classico (RAG)
* 💾 SQL agent che scrive query per te

Preparati a costruire **applicazioni intelligenti multi-tool**! 🚀